In [1]:
import xarray as xr
import numpy as np
import glob
import time

In [2]:
import sys
sys.path.insert(0,'/home/alberta/Work/git/formation_ANNA')
import xscale.xscale

In [3]:
!ls /mnt/alberta/equipes/IGE/meom/workdir/albert/eNATL60/eNATL60-BLBT02-S/1h/3D/

eNATL60-BLBT02_1h_20090630_20090704_gridT_20090630-20090630.nc


In [4]:
datadir = '/mnt/alberta/equipes/IGE/meom/workdir/albert/eNATL60/eNATL60-BLBT02-S/1h/3D/'
dst=xr.open_dataset(datadir+'eNATL60-BLBT02_1h_20090630_20090704_gridT_20090630-20090630.nc') 


In [5]:
def filt(w):
    win_box2D = w.window
    win_box2D.set(window='boxcar', cutoff=80, dim=['x', 'y'], n=[80, 80])
    bw = win_box2D.boundary_weights(drop_dims=[])
    w_LS = win_box2D.convolve(weights=bw)
    w_SS=w-w_LS
    return w_SS


In [6]:
import PowerSpec.PowerSpec as ws


In [7]:
temp=dst.votemper[0,0]
tprime=filt(temp)


In [ ]:
zlat= dst['nav_lat'][600:1200,1800:2400].values
zlon= dst['nav_lon'][600:1200,1800:2400].values
zfull = temp[600:1200,1800:2400].values
zhp = tprime[600:1200,1800:2400].values
zlp = zfull - zhp

In [ ]:
zfull

In [ ]:
import os
import conda

conda_file_dir = conda.__file__
conda_dir = conda_file_dir.split('lib')[0]
proj_lib = os.path.join(os.path.join(conda_dir, 'share'), 'proj')
os.environ["PROJ_LIB"] = proj_lib


In [ ]:
interp = 'basemap'
def compute_spectrum(var):
    data = var.squeeze()
    x_reg,y_reg,data_reg = ws.interpolate(data,zlon,zlat,interp=interp)
    pspec,kstep = ws.wavenumber_spectra(data_reg,x_reg,y_reg)
    return pspec, kstep

%time spectre_full = compute_spectrum(zfull)
%time spectre_hp   = compute_spectrum(zhp)
%time spectre_lp   = compute_spectrum(zlp)


In [ ]:
import matplotlib.pyplot as plt

rad2cyc = 1.E3 / np.pi / 2. 

def nice_spectrum(spectre_full,spectre_hp,spectre_lp):
    fig, axarr = plt.subplots(1,1)
    fig.set_figheight(8)
    fig.set_figwidth(15)
    #
    pspec,kstep = spectre_full
    pspec_lp,kstep_lp = spectre_lp
    pspec_hp,kstep_hp = spectre_hp
    skstep = kstep * rad2cyc
    y_min = 10 ** np.floor(np.log10(pspec.min())-1)
    y_max = 10 ** np.ceil( np.log10(pspec.max())+1)
    axarr.plot(skstep[1:], pspec[1:],'k-', lw=3, label='Full relative vorticity')
    axarr.plot(skstep[1:], pspec_lp[1:],'r-', lw=3, label='Low passed relative vorticity f=0.0125 n=80')
    axarr.plot(skstep[1:], pspec_hp[1:],'b-', lw=3, label='Full - Low passed  f=0.0125 n=80')
    axarr.set_xscale('log')
    axarr.set_yscale('log')
    axarr.set_xbound(1e-5*rad2cyc, 1e-2*rad2cyc)
    axarr.set_ybound(y_min, y_max)
    axarr.set_title('',fontsize=20)
    axarr.grid(True,which='both',ls='-')
    axarr.axis('tight')
    axarr.legend(loc="lower left", fontsize=25)
    axarr.tick_params(labelsize=25)

nice_spectrum(spectre_full,spectre_hp,spectre_lp)
